In [0]:
# Installing packages for initial run 
#!pip install dash
#!pip install chart-studio

In [0]:
import dash 
import dash_core_components as dcc 
import dash_html_components as html 
from dash.dependencies import Input, Output, ClientsideFunction 

import numpy as np 
import pandas as pd 
import datetime 
from datetime import datetime as dt 
import pathlib
 
import os

from google.colab import drive
from google.colab import files

import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [0]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)

server = app.server
app.config.suppress_callback_exceptions = True

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_path = "/content/drive/My Drive/___hertie/2020-spring (1)/rsub-redpill-analysis/data"

In [1]:
# read in data 
df = pd.read_csv(os.path.join(data_path,"dash-input1.csv"))

NameError: ignored

In [0]:
# inputs 1 
subreddit_list = df.iloc[:, 7].unique().tolist() # TheRedPill MGOTW or MensRights
type_list = df.iloc[:, 5].unique().tolist() # hot top new controversial etc. 
attribute_list = df.iloc[:, 3].unique().tolist() # score or number of comments in final dataset 

df["month"] = df["date"].astype(str).str[5:7]
df["year"] = df["date"].astype(str).str[:4]
df["date"] = pd.to_datetime(df.date)
df = df.sort_values(by=["date"])

In [0]:
# subset and recombine data for static ts plots 
df11 = df[df.attribute=='comms_num']
df12 = df11.drop_duplicates(['id2'])

df13 = df[df.attribute=='score']
df14 = df13.drop_duplicates(['id2'])

df1 = df12.append(df14)

In [0]:
# subset of most controversial posts between the three feeds 
condf = df1[df1.post_type=="con"]
condf = condf.drop_duplicates(['id2'])
condf.to_csv('condf.csv')
files.download('condf.csv')

In [0]:
# heatmap df 1 
hm_df = condf.groupby(['month','year']).sum()
hm_df1 = hm_df[['n']]
hm_df1.to_csv('hmdf_con.csv')
files.download('hmdf_con.csv')

In [0]:
#hm_list1 = hm_df1['n'].tolist()
#np.reshape(hm_list1,(12,13)).T

[1, 2, 3]

In [0]:
def generate_heatmap_data(df):
  """
  :return: create heatmaps for number of comments and scores
  """
  list1 = df['n'].tolist()
  z= df['month'].tolist()
  x= df['month'].tolist()
  y= df['year'].tolist()
  heatmap_info = go.Heatmap(z=z, x=x, y=y)
  return heatmap_info

In [0]:
conditions = [df['month'].str.contains('1',na=False),
              df['month'].str.contains('2',na=False),
              df['month'].str.contains('3',na=False),
              df['month'].str.contains('4',na=False),
              df['month'].str.contains('5',na=False),
              df['month'].str.contains('6',na=False),
              df['month'].str.contains('7',na=False),
              df['month'].str.contains('8',na=False),
              df['month'].str.contains('9',na=False),
              df['month'].str.contains('10',na=False),
              df['month'].str.contains('11',na=False),
              df['month'].str.contains('12',na=False)]

choices = ['January','February','March','April','May','June','July','August','September','October','November','December']
df['month1'] = np.select(conditions,choices,default=np.nan)

In [0]:
# input lists 
month_list = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December"
]

year_list = [
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019"
]

In [0]:
def description_card():
    """
    :return: A Div containing dashboard title & descriptions.
    """
    return html.Div(
        id="description-card",
        children=[
            html.H3("Welcome to the Men's Rights Activist Dashboard"),
            html.Div(
                id="intro",
                children="Investigate patterns over time on [BLAH BLAH BLAH ADD TO THIS]",
            ),
        ],
    )

In [0]:
def generate_control_card():
    """
    :return: A Div containing controls for graphs.
    """
    return html.Div(
        id="control-card",
        children=[
            html.P("Select Subreddit Feed"),
            dcc.Dropdown(
                id="subreddit-select",
                options=[{"label": i, "value": i} for i in subreddit_list],
                value=subreddit_list[0],
            ),
            html.P("Select Type"),
            dcc.Dropdown(
                id="type-select",
                options=[{"label": i, "value": i} for i in type_list],
                value=type_list[0],
            ),
            html.P("Select Attribute"),
            dcc.Dropdown(
                id="attribute-select",
                options=[{"label": i, "value": i} for i in type_list],
                value=type_list[0],
            ),
            html.Br(),
            html.Div(
                id="reset-btn-outer",
                children=html.Button(id="reset-btn", children="Reset", n_clicks=0),
            ),
        ],
    )

In [0]:
def generate_heatmap(subreddit, post_type, attribute, reset):
    """
    :param: subreddit: subreddit feed from selection.
    :param: type: type of feed (hot, top, new, controversial).
    :param: attribute: number of comments or score (up votes minus down votes)
    :param: reset (boolean): reset if True.
    :return: plot of # comments or score over time.
    """

    filtered_df = df1[
        (df["subreddit "] == subreddit) & (df["post_type"] == post_type) & (df["attribute"] == attribute)
    ]
    filtered_df = filtered_df.sort_values("date").set_index("date")[
        start:end
    ]

    x_axis = month_list  # Month 
    y_axis = year_list # Year 

    month = ""
    year = ""
    shapes = []

    if hm_click is not None:
        month = hm_click["points"][0]["x"]
        year = hm_click["points"][0]["y"]

        # Add shapes
        x0 = x_axis.index(month) / 12
        x1 = x0 + 1 / 12
        y0 = y_axis.index(year) / 7
        y1 = y0 + 1 / 7

        shapes = [
            dict(
                type="rect",
                xref="paper",
                yref="paper",
                x0=x0,
                x1=x1,
                y0=y0,
                y1=y1,
                line=dict(color="#ff6347"),
            )
        ]

    # Get z value : sum(number of posts or score) based on x, y,

    z = np.zeros((7, 12))
    annotations = []

    for ind_y, year in enumerate(y_axis):
        filtered_year = filtered_df[filtered_df["Year"] == year]
        for ind_x, x_val in enumerate(x_axis):
            sum_of_posts = filtered_year[filtered_year["Month"] == x_val][
                "attribute"
            ].sum()
            z[ind_y][ind_x] = sum_of_posts

            annotation_dict = dict(
                showarrow=False,
                text="<b>" + str(sum_of_posts) + "<b>",
                xref="x",
                yref="y",
                x=x_val,
                y=year,
                font=dict(family="sans-serif"),
            )
            # Highlight annotation text by self-click
            if x_val == month and year == year:
                if not reset:
                    annotation_dict.update(size=15, font=dict(color="#ff6347"))

            annotations.append(annotation_dict)

    # Heatmap
    hovertemplate = "<b> %{y}  %{x} <br><br> %{z} Comments/Upvotes"

    data = [
        dict(
            x=x_axis,
            y=y_axis,
            z=z,
            type="heatmap",
            name="",
            hovertemplate=hovertemplate,
            showscale=False,
            colorscale=[[0, "#caf3ff"], [1, "#2c82ff"]],
        )
    ]

    layout = dict(
        margin=dict(l=70, b=50, t=50, r=50),
        modebar={"orientation": "v"},
        font=dict(family="Open Sans"),
        annotations=annotations,
        shapes=shapes,
        xaxis=dict(
            side="top",
            ticks="",
            ticklen=2,
            tickfont=dict(family="sans-serif"),
            tickcolor="#ffffff",
        ),
        yaxis=dict(
            side="left", ticks="", tickfont=dict(family="sans-serif"), ticksuffix=" "
        ),
        hovermode="closest",
        showlegend=False,
    )
    return {"data": data, "layout": layout}

In [0]:
## set index 
df2 = df1.set_index(['date'])

# subset for setting ts series  
trp_df1 = df2[(df2['subreddit ']=='TheRedPill') & (df2['attribute']=='comms_num')]
trp_df2 = df2[(df2['subreddit ']=='TheRedPill') & (df2['attribute']=='score')]

mgtow_df1 = df2[(df2['subreddit ']=='MGTOW') & (df2['attribute']=='comms_num')]
mgtow_df2 = df2[(df2['subreddit ']=='MGTOW') & (df2['attribute']=='score')]

mr_df1 = df2[(df2['subreddit ']=='MensRights') & (df2['attribute']=='comms_num')]
mr_df2 = df2[(df2['subreddit ']=='MensRights') & (df2['attribute']=='score')]

In [0]:
def generate_ts_series(df): 
  """
  :return: create static time series subplots  
  """
  ts_series = df.loc[:,('n')]
  ts_series = go.Scatter(x=ts_series.index, y=ts_series.values)
  return ts_series 

In [0]:
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("r/TheRedPill","r/MGTOW","r/MensRights"),
                    vertical_spacing=0.12)

fig.append_trace(go.Scatter(
    generate_ts_series(trp_df1),
    name= '# Comments',
    legendgroup="a",
    mode = 'lines',
    marker = {'symbol':'circle', 'color':'#0D0887'}
), row=1, col=1)

fig.append_trace(go.Scatter(
    generate_ts_series(trp_df2),
    name= 'Score',
    legendgroup="b",
    mode = 'lines',
    marker = {'symbol':'circle', 'color':'#E16462'}
), row=1, col=1)

fig.append_trace(go.Scatter(
    generate_ts_series(mgtow_df1),
    name= '# Comments',
    legendgroup="a",
    showlegend = False,
    mode = 'lines',
    marker = {'symbol':'circle', 'color':'#0D0887'}
), row=2, col=1)

fig.append_trace(go.Scatter(
    generate_ts_series(mgtow_df2),
    name= 'Score',
    legendgroup="b",
    showlegend = False,   
    mode = 'lines',
    marker = {'symbol':'circle', 'color':'#E16462'}
), row=2, col=1)

fig.append_trace(go.Scatter(
    generate_ts_series(mr_df1),
    name= '# Comments',
    legendgroup="a",
    showlegend = False,
    mode = 'lines',
    marker = {'symbol':'circle', 'color':'#0D0887'}
), row=3, col=1)

fig.append_trace(go.Scatter(
    generate_ts_series(mr_df2),
    name= 'Score',
    legendgroup="b",
    showlegend = False,
    mode = 'lines',
    marker = {'symbol':'circle', 'color':'#E16462'}
), row=3, col=1)

In [0]:
app.clientside_callback(
    ClientsideFunction(namespace="clientside", function_name="resize"),
    Output("output-clientside", "children"),
    [Input("wait_time_table", "children")] + wait_time_inputs + score_inputs,
)


In [0]:
app.layout = html.Div(
    id="app-container",
    children=[
        # Banner
        html.Div(
            id="banner",
            className="banner",
            children=[html.Img(src=app.get_asset_url("plotly_logo.png"))],
        ),
        # Left column
        html.Div(
            id="left-column",
            className="four columns",
            children=[description_card(), generate_control_card()]
            + [
                html.Div(
                    ["initial child"], id="output-clientside", style={"display": "none"}
                )
            ],
        ),
        # Right column
        html.Div(
            id="right-column",
            className="eight columns",
            children=[
                # Post Frequency Heatmap
                html.Div(
                    id="patient_volume_card",
                    children=[
                        html.B("Patient Volume"),
                        html.Hr(),
                        dcc.Graph(id="npost_hm"),
                    ],
                ),
                # Static time series plots by subreddit 
                html.Div([
                    dcc.Graph(figure=fig)
                ]),
            ],
        ),
    ],
)

In [0]:
@app.callback(
    Output("npost_hm", "figure"),
    [
        Input("subreddit-select", "value"),
        Input("npost_hm", "clickData"),
        Input("type-select", "value"),
        Input("attribute-select", "value"),
        Input("reset-btn", "n_clicks"),
    ],
)
def update_heatmap(subreddit, post_type, attribute, reset):

    reset = False
    # Find which one has been triggered
    ctx = dash.callback_context

    if ctx.triggered:
        prop_id = ctx.triggered[0]["prop_id"].split(".")[0]
        if prop_id == "reset-btn":
            reset = True

    # Return to original hm(no colored annotation) by resetting
    return generate_heatmap(
        subreddit, post_type, attribute, reset
    )
